In [1]:
# general imports
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# load required Sionna components
from sionna.fec.utils import load_parity_check_examples, LinearEncoder, gm2pcm
from sionna.fec.ldpc import LDPCBPDecoder
import sionna

%load_ext autoreload
%autoreload 2
from gnn import * # load GNN functions
from wbp import * # load weighted BP functions

In [2]:
#----- BCH -----
params={
    # --- Code Parameters ---
        "code": "BCH", # (63,45)
    # --- GNN Architecture ----
        "num_embed_dims": 20,
        "num_msg_dims": 20,
        "num_hidden_units": 40,
        "num_mlp_layers": 2,
        "num_iter": 8,
        "reduce_op": "mean",
        "activation": "tanh",
        "clip_llr_to": None,
        "use_attributes": False,
        "node_attribute_dims": 0,
        "msg_attribute_dims": 0,
        "use_bias": False,
}

In [3]:
pcm, k, n, coderate = load_parity_check_examples(pcm_id=1, verbose=True)



encoder = LinearEncoder(pcm, is_pcm=True)


n: 63, k: 45, coderate: 0.714


In [4]:
bp_decoder = LDPCBPDecoder(pcm, num_iter=20, hard_out=False)

tf.random.set_seed(2) # we fix the seed to ensure stable convergence


# init the GNN decoder
gnn_decoder = GNN_BP(pcm=pcm,
                     num_embed_dims=params["num_embed_dims"],
                     num_msg_dims=params["num_msg_dims"],
                     num_hidden_units=params["num_hidden_units"],
                     num_mlp_layers=params["num_mlp_layers"],
                     num_iter=params["num_iter"],
                     reduce_op=params["reduce_op"],
                     activation=params["activation"],
                     output_all_iter=True,
                     clip_llr_to=params["clip_llr_to"],
                     use_attributes=params["use_attributes"],
                     node_attribute_dims=params["node_attribute_dims"],
                     msg_attribute_dims=params["msg_attribute_dims"],
                     use_bias=params["use_bias"])

e2e_gnn = E2EModel(encoder, gnn_decoder, k, n)
e2e_gnn(1, 1.)

e2e_gnn.summary()

train = False # remark: training takes several hours
if train:
    train_gnn(e2e_gnn, params)
else:
    # you can also load the precomputed weights
    load_weights(e2e_gnn, "weights/BCH_precomputed.npy")

2023-12-26 16:05:13.277896: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x14fcd94f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-12-26 16:05:13.277919: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-12-26 16:05:13.412853: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:102] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. binary_source/random_uniform
2023-12-26 16:05:13.433493: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-26 16:05:13.594767: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator mapper/assert_greater_equal/Assert/Assert
2023-12-26 16:05:13.594791: W tensorflow/compiler/tf2xla/kernels/assert_o

Model: "e2e_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 binary_source (BinarySourc  multiple                  0         
 e)                                                              
                                                                 
 mapper (Mapper)             multiple                  0         
                                                                 
 demapper (Demapper)         multiple                  0         
                                                                 
 awgn (AWGN)                 multiple                  0         
                                                                 
 gnn_bp (GNN_BP)             multiple                  9640      
                                                                 
 linear_encoder (LinearEnco  multiple                  0         
 der)                                                    

2023-12-26 16:05:14.576305: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [5]:
gnn_dec_temp = GNN_BP(pcm=pcm,
                          num_embed_dims=params["num_embed_dims"],
                          num_msg_dims=params["num_msg_dims"],
                          num_hidden_units=params["num_hidden_units"],
                          num_mlp_layers=params["num_mlp_layers"],
                          num_iter=6,
                          reduce_op=params["reduce_op"],
                          activation=params["activation"],
                          output_all_iter=True,
                          clip_llr_to=params["clip_llr_to"],
                          use_attributes=params["use_attributes"],
                          node_attribute_dims=params["node_attribute_dims"],
                          msg_attribute_dims=params["msg_attribute_dims"],
                          use_bias=params["use_bias"])

model = E2EModel(encoder, gnn_dec_temp, k, n)
model(1,1.) # init model
# copy weights from trained decoder
model._decoder.set_weights(gnn_decoder.get_weights())

2023-12-26 16:05:16.310342: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator mapper_1/assert_greater_equal/Assert/Assert
2023-12-26 16:05:16.310367: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator awgn_1/assert_greater_equal/Assert/Assert
2023-12-26 16:05:16.310376: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator awgn_1/assert_less_equal/Assert/Assert
2023-12-26 16:05:16.310436: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator awgn_1/assert_greater_equal_1/Assert/Assert


In [6]:

# pruning function
import pandas as pd
from tensorflow.python.ops.script_ops import numpy_function
import random

llr_embed = e2e_gnn._decoder._llr_inv_embed

msg_mlp_vn = e2e_gnn._decoder.update_h_vn._msg_mlp._layers
embed_mlp_vn = e2e_gnn._decoder.update_h_vn._embed_mlp._layers

msg_mlp_cn = e2e_gnn._decoder.update_h_cn._msg_mlp._layers
embed_mlp_cn = e2e_gnn._decoder.update_h_cn._embed_mlp._layers

# pruning function 구현
def pruner_mlp(mlp, K): # K 에 맞는 비율로 pruning을 진행한다.
    for layer_no in range(len(mlp)):
        all_weights = {}
        layer_weights = (pd.DataFrame(mlp[layer_no].get_weights()[0]).stack()).to_dict()
        layer_weights = {(layer_no, k[0], k[1]): v for k, v in layer_weights.items()}
        all_weights.update(layer_weights)

        all_weights_sorted = {k: v for k, v in sorted(all_weights.items(), key=lambda item: abs(item[1]))}

        total_no_weights = len(all_weights_sorted)

        if (layer_no == 0):
            pruning_percent = K[layer_no]
            print(layer_no, '번째 layer')
            new_weights = mlp[layer_no].get_weights()
            prune_fraction = pruning_percent/100 # K에서 지정한 비율만큼 pruning
            number_of_weights_to_be_pruned = int(prune_fraction*total_no_weights)
            print('number_of_weights_to_be_pruned : ', number_of_weights_to_be_pruned)
            weights_to_be_pruned = {k: all_weights_sorted[k] for k in list(all_weights_sorted)[: number_of_weights_to_be_pruned]}

            for k, v in weights_to_be_pruned.items():
                if (k[0]==0):
                    new_weights[0][k[1], k[2]]=0 # weights_to_be_pruned에 저장된 인덱스의 가중치를 0으로 만들어 준다.
                elif (k[0]==1):
                    new_weights[0][k[1], k[2]]=0
                else:
                    print('error')

            mlp[layer_no].set_weights(new_weights)

        elif (layer_no == 1):
            pruning_percent = K[layer_no]
            print(layer_no, '번째 layer')
            new_weights = mlp[layer_no].get_weights()
            prune_fraction = pruning_percent/100 # K에서 지정한 비율만큼 pruning
            number_of_weights_to_be_pruned = int(prune_fraction*total_no_weights)
            print('number_of_weights_to_be_pruned : ', number_of_weights_to_be_pruned)
            weights_to_be_pruned = {k: all_weights_sorted[k] for k in list(all_weights_sorted)[: number_of_weights_to_be_pruned]}

            for k, v in weights_to_be_pruned.items():
                if (k[0]==0):
                    new_weights[0][k[1], k[2]]=0 # weights_to_be_pruned에 저장된 인덱스의 가중치를 0으로 만들어 준다.
                elif (k[0]==1):
                    new_weights[0][k[1], k[2]]=0
                else:
                    print('error')
            mlp[layer_no].set_weights(new_weights)
        else:
            print('error')

def random_pruner_mlp(mlp, K): # random pruning을 진행한다.
    all_weights = {}

    for layer_no in range(len(mlp)-1):
        layer_weights = (pd.DataFrame(mlp[layer_no].get_weights()[0]).stack()).to_dict()
        layer_weights = {(layer_no, k[0], k[1]): v for k, v in layer_weights.items()}
        all_weights.update(layer_weights)

    all_weights_sorted = {k: v for k, v in sorted(all_weights.items(), key=lambda item: abs(item[1]))}

    print(all_weights_sorted)
    total_no_weights = len(all_weights_sorted)

    for pruning_percent in K:
        for layer_no in range(len(mlp) -1):
            new_weights = mlp[layer_no].get_weights()

            prune_fraction = pruning_percent / 100 # K에서 지정한 비율만큼 pruning
            number_of_weights_to_be_pruned = int(prune_fraction*total_no_weights)
            # random pruning에서 중요한 부분
            random_indices = np.random.choice(total_no_weights, size=number_of_weights_to_be_pruned, replace=False)
            random_weights_to_be_pruned = [list(layer_weights.keys())[i] for i in random_indices]

            for k in random_weights_to_be_pruned:
                new_weights[k[0]][k[1], k[2]]=0 # weights_to_be_pruned에 저장된 인덱스의 가중치를 0으로 만들어 준다.

            mlp[layer_no].set_weights(new_weights)


 # pruning percentage
embed_pruning_per = [20,2]
msg_pruning_per = [10, 2]

pruner_mlp(embed_mlp_vn, embed_pruning_per)
pruner_mlp(msg_mlp_vn, msg_pruning_per)

pruner_mlp(embed_mlp_cn, embed_pruning_per)
pruner_mlp(msg_mlp_cn, msg_pruning_per)

0 번째 layer
number_of_weights_to_be_pruned :  320
1 번째 layer
number_of_weights_to_be_pruned :  16
0 번째 layer
number_of_weights_to_be_pruned :  160
1 번째 layer
number_of_weights_to_be_pruned :  16
0 번째 layer
number_of_weights_to_be_pruned :  320
1 번째 layer
number_of_weights_to_be_pruned :  16
0 번째 layer
number_of_weights_to_be_pruned :  160
1 번째 layer
number_of_weights_to_be_pruned :  16


In [7]:
def decoding_process(frame, decoder_type, eb_no):
    flat_frame = frame.reshape(-1)
    binary_data = ''.join(format(pixel, '08b') for pixel in flat_frame)

    binary_data = list(binary_data)
    b = np.array(binary_data) # numpy 배열 형식으로 변환

    b = [float(element) for element in b] # 각 value -> float형으로 cast
    b_np = np.reshape(b, (-1, 45)) # 45개씩 처리

    bit_len = len(b_np)
    bch_code = encoder(b_np) # 이진 데이터를 BCH 코드로 인코딩

    b_pad = tf.concat([bch_code, tf.zeros([bit_len , 1])], axis=1)

    mapper = Mapper("qam", 2)
    demapper = Demapper("app", "qam", 2)
    channel = AWGN()
    no = ebnodb2no(eb_no, 2, 0.714) # 잡음 생성 부분 (신호대잡음비, bit per symbol, coderate) 각각 설정하여 잡음 생성

    x = mapper(b_pad) # 심볼 생성(복소수로 표현)
    y = channel([x, no]) # 채널 통과
    llr = demapper([y, no]) # 심볼 복호
    llr = llr[:,:-1]
  
    # BP, GNN(basic GNN, prune GNN) 사용하고자 하는 타입 설정
    if decoder_type == 'BP':
        llr = bp_decoder(llr)
        c_hat=[]
        for i in range(len(llr)):
            c_tmp = tf.cast(tf.greater(llr[i], 0), tf.float32)
            c_hat.append(c_tmp.numpy())
    elif decoder_type == 'GNN':
        llr = model._decoder(llr)
        c_hat = []
        for i in range(len(llr[-1])):
            c_tmp = tf.cast(tf.greater(llr[-1][i], 0), tf.float32)
            c_hat.append(c_tmp.numpy())
    else:
        print('unavaliable decoder')

    r = []
    for i in range(0, 122880):
        result = np.concatenate((c_hat[i][45:63], c_hat[i][18:45])) # -- 사용한 생성 행렬의 구조에 맞게 코드워드로부터 메세지 추출 --
        r.append(result)
    r = [int(bit) for row in r for bit in row] # 모두 int형으로 변환

    decoded_bit = ''.join(str(element) for element in r) # 한 줄의 비트열로 변환

    # 한 줄의 비트열을 8개씩 쪼개서, 하나의 RGB값으로 변환해주는 코드
    bytes_array = np.array([int(decoded_bit[i:i+8], 2) for i in range(0, len(decoded_bit), 8)], dtype=np.uint8)
    # RGB 값을 배열 형태로 변환해준다. 이미지로 복원하기 위해 필요한 코드
    bites_array = bytes_array.reshape((360, 640, 3))

    return bites_array


In [15]:
import cv2

video_file = 'video1.mp4'

import cv2

try:
    cap = cv2.VideoCapture(video_file)
    print('동영상 읽기 성공')
except:
    print('동영상 읽기 실패')

vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) #(200, 400)
vid_fps = cap.get(cv2.CAP_PROP_FPS)
frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('총 Frame 갯수:', frame_cnt, 'FPS:', round(vid_fps), 'Frame 크기:', vid_size)

output_path = 'output_video.mp4'
fps = 24
#out = cv2.VideoWriter(output_path,cv2.VideoWriter_fourcc(*'XVID'),fps, (854,480))

decoder_type = 'BP' # ------------BP, GNN 설정 !!--------------
eb_no= 7.0
count = 0

while True:
    # 동영상에서 이미지를 프레임 단위로 읽어옴
    ret, frame = cap.read()
    print(count, "번째 실행 중")
    # if not ret : # ret이 False일때 (파일에 문제가 있거나 동영상이 끝났을때)
    if count == 720:
        print('이미지 읽기 실패 또는 모두 읽음')
        # 비디오 종료
        cap.release()
        # 윈도우 실행창 종료
        cv2.destroyAllWindows()
        # 반복문 종료
        break


    frame = cv2.resize(frame, (640, 360)) #  360p
    if count >= 0:
        after_frame = decoding_process(frame, decoder_type, eb_no)
        filepath = 'output_6.0_BP_360'
        cv2.imwrite(filepath + "/frame%d.jpg" %count, after_frame)
        print('Saved frame number :', str(int(cap.get(1))))
    
    count+=1
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

동영상 읽기 성공
총 Frame 갯수: 213 FPS: 30 Frame 크기: (1280, 720)
0 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 1
1 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 2
2 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 3
3 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 4
4 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 5
5 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 6
6 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 7
7 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 8
8 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 9
9 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 10
10 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 11
11 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 12
12 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 13
13 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 14
14 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 15
15 

Saved frame number : 127
127 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 128
128 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 129
129 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 130
130 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 131
131 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 132
132 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 133
133 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 134
134 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 135
135 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 136
136 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 137
137 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 138
138 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 139
139 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 140
140 번째 실행 중
122880
5529600
(360, 640, 3)
Saved frame number : 141
141 번째 실행 중
122880
5529600
(360, 640, 3)
Saved fram

error: OpenCV(4.8.1) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
import cv2

img_file = 'monalisa.jpg'
decoder_type = 'BP'
eb_no = 7.0

img = cv2.imread(img_file)
img = cv2.resize(img, (640, 360))
img = decoding_process(img, decoder_type, eb_no)

cv.imwrite(decoder_type+"_"+eb_no+"_monalisa.jpg")
cv2.destroyAllWindows()

In [27]:
import cv2
import time
import os
import re

path = 'output_7.0_prunedGNN_360_cat'
#os.remove(path+"/.DS_store")
def get_files(path):
    files = os.listdir(path)
    
    print(files)
    list_files = []
    
    def digitSum(n): # 파일의 숫자대로 정렬을 위한 함수 ex) frame64, frame32가 있을 때 64, 32를 찾아 배열    
        n = int(re.findall("\d+", n)[0])
        return n

    files = sorted(files, reverse=False, key=digitSum) # 실제 정렬 함수

    if len(files) > 0: # 정렬된 파일 리스트를 경로를 붙여 반환한다.
        for f in files:
            fullpath = path + '/' + f
            list_files.append(fullpath)
    return list_files 

image_files = get_files(path) # 정렬된 파일 경로 리스트
img = cv2.imread(image_files[0]) # 각 이미지 파일을 읽는다
height,width,channel = img.shape # 이미지 높이, 넓이, 채널 정보 
fps = 18 # 초당 프레임

fourcc = cv2.VideoWriter_fourcc(*'mp4v') # 영상 형식 설정
writer = cv2.VideoWriter(path + '.mp4', fourcc, fps, (width, height)) # 영상 작성 선언

for file in image_files:
    img = cv2.imread(file)
    writer.write(img)
    # ESC키 누르면 중지
    if cv2.waitKey(25) == 27:
        break

writer.release()
cv2.destroyAllWindows()

['frame193.jpg', 'frame187.jpg', 'frame85.jpg', 'frame91.jpg', 'frame150.jpg', 'frame144.jpg', 'frame46.jpg', 'frame52.jpg', 'frame178.jpg', 'frame53.jpg', 'frame179.jpg', 'frame47.jpg', 'frame145.jpg', 'frame151.jpg', 'frame90.jpg', 'frame84.jpg', 'frame186.jpg', 'frame192.jpg', 'frame184.jpg', 'frame190.jpg', 'frame92.jpg', 'frame86.jpg', 'frame147.jpg', 'frame79.jpg', 'frame153.jpg', 'frame51.jpg', 'frame45.jpg', 'frame44.jpg', 'frame50.jpg', 'frame78.jpg', 'frame152.jpg', 'frame146.jpg', 'frame87.jpg', 'frame93.jpg', 'frame191.jpg', 'frame185.jpg', 'frame208.jpg', 'frame97.jpg', 'frame83.jpg', 'frame181.jpg', 'frame195.jpg', 'frame54.jpg', 'frame40.jpg', 'frame68.jpg', 'frame142.jpg', 'frame156.jpg', 'frame157.jpg', 'frame69.jpg', 'frame41.jpg', 'frame55.jpg', 'frame194.jpg', 'frame180.jpg', 'frame82.jpg', 'frame96.jpg', 'frame209.jpg', 'frame80.jpg', 'frame94.jpg', 'frame196.jpg', 'frame182.jpg', 'frame43.jpg', 'frame169.jpg', 'frame57.jpg', 'frame155.jpg', 'frame141.jpg', 'frame1

In [ ]:
binary_data = list(binary_data)

b = np.array(binary_data)

# 각 value -> float형으로 cast
b = [float(element) for element in b]
print('before : ', len(b))
# 나머지 0으로 채우는 코드

# b += [0.0]*15

print('after : ', len(b))
# 45개씩 끊어서 정리
b_np = np.reshape(b, (-1, 45))

print(len(b_np))

bit_len = len(b_np)
bch_code = encoder(b_np) # 이진 데이터를 BCH 코드로 인코딩

In [ ]:
b_pad = tf.concat([bch_code, tf.zeros([bit_len , 1])], axis=1)

In [ ]:
mapper = Mapper("qam", 2)
demapper = Demapper("app", "qam", 2)
channel = AWGN()
no = ebnodb2no(7.0, 2, 0.714) #
x = mapper(b_pad)
y = channel([x, no])

llr = demapper([y, no])
llr = llr[:,:-1]

In [ ]:
llr = bp_decoder(llr)
c_hat=[]
for i in range(len(llr)):
  c_tmp = tf.cast(tf.greater(llr[i], 0), tf.float32)
  c_hat.append(c_tmp.numpy())

"""
llr = model._decoder(llr)

c_hat = []
for i in range(len(llr[-1])):
  c_tmp = tf.cast(tf.greater(llr[-1][i], 0), tf.float32)
  c_hat.append(c_tmp.numpy())
"""

In [ ]:
r = []
for i in range(0, 218624):
  result = np.concatenate((c_hat[i][45:63], c_hat[i][18:45]))
  r.append(result)
r = [int(bit) for row in r for bit in row]

In [ ]:
MonaLisa_bit = ''.join(str(element) for element in r)
print(len(MonaLisa_bit))

bytes_array = np.array([int(MonaLisa_bit[i:i+8], 2) for i in range(0, len(MonaLisa_bit), 8)], dtype=np.uint8)

# 배열을 이미지로 변환

bites_array = bytes_array.reshape((480, 854, 3))
print(bites_array.shape)

# 이미지로 저장

image = Image.fromarray(bites_array)
print(image.size)
image.save('MonaLisa_bit.png')
